# Mission Simulation: Time-Series Link Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jman4162/opensatcom/blob/main/notebooks/02_mission_simulation.ipynb)

Simulate a satellite pass and analyze link margin, elevation, and availability over time.

In [ ]:
# Install opensatcom (uncomment for Colab)
# !pip install -q opensatcom

In [ ]:
import numpy as np
from opensatcom.core.models import *
from opensatcom.antenna.parametric import ParametricAntenna
from opensatcom.propagation import FreeSpacePropagation
from opensatcom.geometry.slant import slant_range_m
from opensatcom.world.sim import SimpleWorldSim
from opensatcom.world.providers import PrecomputedTrajectory, StaticEnvironmentProvider

## 1. Define Link Setup

In [ ]:
link_inputs = LinkInputs(
    tx_terminal=Terminal("GEO-Sat", 0.0, 0.0, 35_786_000.0),
    rx_terminal=Terminal("Ground", 38.9, -77.0, 0.0, system_noise_temp_k=290.0),
    scenario=Scenario(
        name="Ku-band DL", direction="downlink",
        freq_hz=12e9, bandwidth_hz=36e6,
        polarization="RHCP", required_metric="ebn0_db", required_value=3.0,
    ),
    tx_antenna=ParametricAntenna(gain_dbi=36.0),
    rx_antenna=ParametricAntenna(gain_dbi=38.0),
    propagation=FreeSpacePropagation(),
    rf_chain=RFChainModel(tx_power_w=100.0, tx_losses_db=1.5, rx_noise_temp_k=75.0),
)

## 2. Generate Synthetic Pass Trajectory

In [ ]:
n_steps = 200
times = np.linspace(0, 600, n_steps)
elev = np.concatenate([
    np.linspace(5.0, 80.0, n_steps // 2),
    np.linspace(80.0, 5.0, n_steps - n_steps // 2),
])
az = np.linspace(120, 240, n_steps)
range_arr = np.array([
    slant_range_m(0.0, 35_786_000.0, e) for e in elev
])

traj = PrecomputedTrajectory.from_arrays(times, elev, az, range_arr)
print(f"Pass duration: {times[-1] - times[0]:.0f}s, {n_steps} steps")

## 3. Run WorldSim

In [ ]:
ops = OpsPolicy(min_elevation_deg=10.0, max_scan_deg=60.0)
env = StaticEnvironmentProvider(PropagationConditions())

sim = SimpleWorldSim()
result = sim.run(link_inputs, traj, ops, env)

print("\nSimulation Summary:")
for key, val in result.summary.items():
    print(f"  {key}: {val:.4f}")

## 4. Visualize Results

In [ ]:
from opensatcom.viz.timeline import plot_link_margin_timeline, plot_elevation_profile

fig = plot_link_margin_timeline(result.times_s, result.margin_db, result.outages_mask)
fig.show()

In [ ]:
fig = plot_elevation_profile(result.times_s, result.elev_deg)
fig.show()

In [ ]:
from opensatcom.viz.statistical import plot_margin_distribution

fig = plot_margin_distribution(result.margin_db)
fig

---

**Next:** See `03_multibeam_payload.ipynb` for multi-beam capacity analysis.